## limit to only inpatients

In [0]:
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import countDistinct, col, row_number
from pyspark.sql.window import Window   
import re

In [0]:
patdemo=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.patdemo")
paticd=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.paticd_diag")
patbill=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.patbill")
chgmstr=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.chgmstr")
distat=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.disstat")
labres=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.lab_res")
sdoh=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.pat_sdoh")
patproc=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.paticd_proc")
ccsrpcc=spark.read.table("edav_prd_cdh.cdh_reference_data.ccsr_pcs_codelist")
ccsrdx=spark.read.table("edav_prd_cdh.cdh_reference_data.ccsr_dx_codelist")
patcpt=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.patcpt")
cpt_desc=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.cptcode")
prov=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.providers")
prov_zip=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.prov_digit_zip")
vitals=spark.read.table("edav_prd_cdh.cdh_premier_1cdp_feasibility.vitals")


In [0]:
provider=prov.join(prov_zip, on='PROV_ID', how='inner').select('PROV_ID', 'PROV_ZIP', 'TEACHING', 'BEDS_GRP', 'URBAN_RURAL')

#provider.display()

In [0]:
#Limiting data to facilities that have provide clinic information and patients with age greater than 17 the final table clin_patdemo will replace patdemo.

clin_fac=patdemo.filter((patdemo.PAT_TYPE == 34) & (patdemo.AGE >=18)).select(patdemo.PAT_TYPE.alias('PAT_TYPE_CLINIC'), 'PROV_ID').distinct()


clin_patdemo=patdemo.join(clin_fac, on='PROV_ID', how='inner')

#clin_patdemo.display()

In [0]:
# filtering paticd and lab res tables for covid diagnosis and lab. filtering covid cohort to patients seen in either clinic, hospital or ED, finding how many covid diagnosis each patient has and determining first covid 19 visit.

covid=(paticd.filter((paticd.ICD_CODE=='U07.1') & (paticd.ICD_PRI_SEC.isin(['P', 'S']))).select('PAT_KEY', paticd.ICD_CODE.alias('COV_DIAG')))

cov_lab = labres.filter(
    labres.LAB_TEST_DESC.isin([
        'SARS coronavirus 2 RNA:PrThr:Pt:Respiratory:Ord:Probe.amp.tar',
        'SARS coronavirus 2 RNA:PrThr:Pt:XXX:Ord:Probe.amp.tar'
    ]) & 
    (labres.OBSERVATION.isin(["positive"]))
).select('PAT_KEY', labres.OBSERVATION.alias('COV_LAB'))

cov_pat=(covid.join(cov_lab, on='PAT_KEY', how='outer'))



covid_cohort = (
    clin_patdemo.join(cov_pat, on='PAT_KEY', how='inner')
    .join(provider, on='PROV_ID', how='left')
    .join(sdoh, on='PAT_KEY', how='left')
    .select('MEDREC_KEY', 'PAT_KEY', 'ADMIT_DATE', 'DISCHARGE_DATE', 
            'PROV_ID', 'I_O_IND', 'PAT_TYPE', 'POINT_OF_ORIGIN', 
            'ADM_TYPE', 'DISC_STATUS', 'AGE', 'GENDER', 
            'RACE', 'HISPANIC_IND', 'STD_PAYOR', 'LOS',
            'MART_STATUS', 'PROV_ZIP', 'TEACHING', 'BEDS_GRP', 'URBAN_RURAL',
            'SVI_RPL_THEMES_ALL_CTGY', 'ACS_AVG_HH_SIZE_CTGY', 'ACS_GINI_INDEX_CTGY',
            'ACS_MEDIAN_HH_INCOME_CTGY', 'ACS_EDUCATION_WGT_CTGY',
             F.when((clin_patdemo['PAT_TYPE'] == 8) & (clin_patdemo['I_O_IND'] == 'I'),1).otherwise(0).alias('INP'))
            .filter((clin_patdemo['PAT_TYPE'] == 8) & (clin_patdemo['I_O_IND'] == 'I') 
                    & (clin_patdemo.ADMIT_DATE >= '2024-06-01') & (clin_patdemo.ADMIT_DATE <= '2024-12-31')).distinct()) 
   

cov_visit = covid_cohort.groupBy('MEDREC_KEY').count()

cov_visit = cov_visit.withColumnRenamed('count', 'COV_COUNT')

cov_cohort_1 = covid_cohort.join(cov_visit, on='MEDREC_KEY', how='left')

fir_cov_1=cov_cohort_1.groupBy('MEDREC_KEY').agg(F.min("ADMIT_DATE").alias('FIRST_COV'))

fir_cov_1=fir_cov_1.join(cov_cohort_1, on='MEDREC_KEY', how='left')\
    .select('MEDREC_KEY', 'PAT_KEY', fir_cov_1.FIRST_COV, 'ADMIT_DATE', 'DISCHARGE_DATE', 
            'PROV_ID', 'I_O_IND', 'PAT_TYPE', 'POINT_OF_ORIGIN', 
            'ADM_TYPE', 'DISC_STATUS', 'AGE', 'GENDER', 
            'RACE', 'HISPANIC_IND', 'STD_PAYOR', 'LOS',
            'MART_STATUS', 'PROV_ZIP', 'TEACHING', 'BEDS_GRP', 'URBAN_RURAL',
            'SVI_RPL_THEMES_ALL_CTGY', 'ACS_AVG_HH_SIZE_CTGY', 'ACS_GINI_INDEX_CTGY',
            'ACS_MEDIAN_HH_INCOME_CTGY', 'ACS_EDUCATION_WGT_CTGY', 'INP')\
    .filter(cov_cohort_1['ADMIT_DATE'] == fir_cov_1['FIRST_COV']).orderBy('MEDREC_KEY').distinct() #limiting to just initial covid visit during period


fir_cov_1.orderBy('MEDREC_KEY').display()


#remove shortest los of stay when patients have the 2 or more same admission date.

In [0]:
# Define window partitioned by ID, ordered by LOS descending
window_spec = Window.partitionBy("MEDREC_KEY").orderBy(col("LOS").desc())

# Add row number within each partition
ranked_df = fir_cov_1.withColumn("rn", row_number().over(window_spec))

# Filter to only the top row per ID (i.e., the row with max LOS)
fir_cov = ranked_df.filter(col("rn") == 1).drop("rn")

In [0]:
display(fir_cov)

In [0]:
cov_count = fir_cov.select('MEDREC_KEY').distinct().count()

print(cov_count)

**PRIOR VISIT & FOLLOW UP**

In [0]:
#determining who had a visit within the year prior (date and patkey PR) to their first covid visit

patdemo_so=patdemo.select('MEDREC_KEY',  patdemo.PAT_KEY.alias('PAT_KEY_PR'), patdemo.ADMIT_DATE.alias('ADMIT_DATE_PR')).filter((patdemo.PAT_TYPE.isin(8, 28, 34)))

loc_bac=fir_cov.join(patdemo_so, on='MEDREC_KEY', how='inner').select('MEDREC_KEY', 'PAT_KEY', 'ADMIT_DATE', 'PAT_KEY_PR', 'ADMIT_DATE_PR', 'FIRST_COV', (F.col("FIRST_COV") - F.lit(365)).alias('YEAR_PRIOR'))

prior_vis=loc_bac.select('MEDREC_KEY', 'PAT_KEY_PR', 'ADMIT_DATE_PR', 'YEAR_PRIOR',
 F.when((loc_bac['ADMIT_DATE_PR'] >= loc_bac['YEAR_PRIOR']) & (loc_bac['ADMIT_DATE_PR'] < loc_bac['FIRST_COV']), 1).otherwise(0).alias('PRIOR_YR')).orderBy('MEDREC_KEY')

prior_vis_1=prior_vis.filter(prior_vis['PRIOR_YR'] == 1).distinct()

#prior_vis_1.orderBy('MEDREC_KEY') .show()

In [0]:
pr_count = prior_vis_1.select('MEDREC_KEY').distinct().count()

print(pr_count)

In [0]:
#limiting our cohort to those with a first covid visit between 2022-01-01 and 2023-12-31 and labeling any inpatient covid visits

cov_cohort_fin=fir_cov.join(prior_vis_1, on='MEDREC_KEY', how='inner') 
  #  .filter((fir_cov.FIRST_COV >= '2022-01-01') & (fir_cov.FIRST_COV <= '2023-12-31'))


#Inpat=Cov_Cohort_2.groupBy('MEDREC_KEY').agg(F.max('INP').alias('INPATIENT'))


#cov_cohort_fin=Cov_Cohort_2.join(Inpat, on='MEDREC_KEY', how='inner')


#display(Cov_Cohort_fin)

In [0]:
fir_count = cov_cohort_fin.select('MEDREC_KEY').distinct().count()

print(fir_count)

In [0]:
#determing follow up patients and limiting the cohort to follow up between 91 and FO65 days

patdemo_fo=patdemo.select('MEDREC_KEY',  patdemo.PAT_KEY.alias('PAT_KEY_FO'), patdemo.ADMIT_DATE.alias('ADMIT_DATE_FO')).filter((patdemo.PAT_TYPE.isin(8, 28, 34)))

loc_fo=cov_cohort_fin.join(patdemo_fo, on='MEDREC_KEY', how='inner').select('MEDREC_KEY', 'PAT_KEY', 'ADMIT_DATE', 'PAT_KEY_FO', 'ADMIT_DATE_FO', 'FIRST_COV', (F.col("FIRST_COV") + F.lit(365)).alias('YEAR_FO'))

fo_vis=loc_fo.select('MEDREC_KEY', 'PAT_KEY_FO', 'ADMIT_DATE_FO', 'YEAR_FO', 'FIRST_COV',
 F.when((loc_fo['ADMIT_DATE_FO'] <= loc_fo['YEAR_FO']) & (loc_fo['ADMIT_DATE_FO'] > loc_fo['FIRST_COV']), 1).otherwise(0).alias('FO_YR')).orderBy('MEDREC_KEY')
   

fo_vis_1=fo_vis.filter(fo_vis['FO_YR'] == 1)\
    .withColumn('FO_TIME', F.datediff(fo_vis['ADMIT_DATE_FO'], fo_vis['FIRST_COV']))\
    .distinct()

#fo_vis_1.orderBy('MEDREC_KEY') .show()


prior_fo=cov_cohort_fin.join(fo_vis_1, on='MEDREC_KEY', how='inner').drop(fo_vis_1.FIRST_COV).filter((fo_vis_1['FO_TIME'] > 90) & ((fo_vis_1['FO_TIME'] <= 365))).distinct()


#prior_fo.orderBy('MEDREC_KEY') .display()
                                                                                                                                              

In [0]:
prior_fo.display()

In [0]:
prior_vis_1.display()

In [0]:
fo_vis_1.display()

In [0]:
fo_count = prior_fo.select('MEDREC_KEY').distinct().count()

print(fo_count)

chargemaster procedures

In [0]:
charge_all = patbill.join(chgmstr, on='STD_CHG_CODE',how='inner')
              
prior_fo_chg = prior_fo.join(charge_all, on='PAT_KEY', how='left')


#display(prior_fo_chg)

In [0]:
clin_sum =prior_fo_chg.groupBy('CLIN_SUM_CODE','CLIN_SUM_DESC') \
                   .agg(countDistinct('MEDREC_KEY').alias('distinct_medrec_count'))

#display(clin_sum)

In [0]:
df_with_cumsum = (
    clin_sum.withColumn('percent', F.col('distinct_medrec_count') / F.lit(fo_count)))

#df_with_cumsum.display()

In [0]:
# Filter to keep only rows within top 95% of cumulative total
clin_sum_inc =['SMOKING DETERRENTS TOPICAL', 'RESPIRATORY VENTILATION SERVICES', 'BRONCHODILATORS GEN ORAL', 'ANTIVIRALS, OTHER ORAL', 'IMMUNOSUP/TRANSPLANT AGENTS ORAL', 'VACCINES, PARENTERAL', 'SURGERY RESPIRATORY LARYNX', 'SURGERY RESPIRATORY LUNG/PLEURA', 'IMMUNOMODULATORS PARENTERAL', 'SURGERY RESPIRATORY TRACHEA/BRONCHI', 'COVID-19 VISITS/TRACKING', 'RESPIRATORY, MISC OTHER', 'RESPIRATORY SUCTIONING PROCEDURES', 'VACCINES, OTHER', 'RESPIRATORY, MISC ORAL', 'SMOKING DETERRENTS ORAL', 'RESPIRATORY EQUIPMENT SERVICES', 'OXYGEN THERAPY SERVICES', 'IMMUNOSUP/TRANSPLANT AGENTS PARENTERAL', 'SMOKING DETERRENTS OTHER', 'IMMUNOSUP/TRANSPLANT AGENTS OTHER', 'BLOOD PRODUCT COVID19 PROCESSING & ADMINISTRATION', 'BLOOD PRODUCTS COVID19', 'EMERGENCY SERVICES COVID19', 'IMMUNOMODULATORS ORAL', 'ANTIVIRALS, OTHER OTHER', 'IMMUNOLOGIC AGENTS, MISC PARENTERAL', 'IMMUNOLOGIC AGENTS, MISC OTHER']

df_top_95 = df_with_cumsum.filter((F.col('percent') >= 0.05) | (F.col('CLIN_SUM_DESC').isin(clin_sum_inc)))

df_top_95.display()

In [0]:
chg_red=chgmstr.join(df_top_95, on='CLIN_SUM_CODE', how='inner')\
    .select('STD_CHG_CODE', 'CLIN_SUM_CODE', df_top_95.CLIN_SUM_DESC)


chg_red_bil = patbill.join(chg_red, on='STD_CHG_CODE', how='inner')\
    .select('PAT_KEY', 'STD_CHG_CODE', 'CLIN_SUM_DESC')

chg_red_cov=prior_fo.join(chg_red_bil, on='PAT_KEY', how='left')\
    .select('PAT_KEY', 'CLIN_SUM_DESC',
             F.when(F.col('CLIN_SUM_DESC')!='NULL', 1).otherwise(0).alias('CHARGE')).distinct()

#chg_red_cov.display()

In [0]:
trans_chg_cov = chg_red_cov.groupBy("PAT_KEY").pivot("CLIN_SUM_DESC").agg(F.first("CHARGE"))
trans_chg_cov = trans_chg_cov.fillna(0)
#trans_chg_cov.display()

In [0]:
chg_comb = trans_chg_cov.select('PAT_KEY',
    F.when(
        (F.col('SMOKING DETERRENTS TOPICAL') == 1) |
        (F.col('SMOKING DETERRENTS ORAL') == 1),
        #|(F.col('SMOKING DETERRENTS OTHER') == 1),
        1
    ).otherwise(0).alias('SMOKING DETERRENTS'),
   
   F.when(
        (F.col('ANTIVIRALS, OTHER ORAL') == 1) |
        #(F.col('ANTIVIRALS, OTHER OTHER') == 1) |
        (F.col('ANTIVIRALS, OTHER PARENTERAL') == 1),
        1
    ).otherwise(0).alias('ANTIVIRALS'),
   
   F.when(
        (F.col('IMMUNOSUP/TRANSPLANT AGENTS ORAL') == 1) |
        (F.col('IMMUNOMODULATORS PARENTERAL') == 1) |
        (F.col('IMMUNOSUP/TRANSPLANT AGENTS PARENTERAL') == 1) |
       # (F.col('IMMUNOSUP/TRANSPLANT AGENTS OTHER') == 1) |
        (F.col('IMMUNOMODULATORS ORAL') == 1) 
       # | (F.col('IMMUNOLOGIC AGENTS, MISC OTHER') == 1) |
        #(F.col('IMMUNOLOGIC AGENTS, MISC PARENTERAL') == 1)
        ,
        1
    ).otherwise(0).alias('IMMUNOSUP/IMMUNOMODULATORS'),
  
   F.when(
        (F.col('VACCINES, PARENTERAL') == 1) |
        (F.col('VACCINES, OTHER') == 1),
        1
    ).otherwise(0).alias('VACCINES'),
    
   F.when(
        (F.col('BLOOD PRODUCT PROCESSING & ADMINISTRATION') == 1) |
        #(F.col('BLOOD RELATED PRODUCTS PARENTERAL') == 1) |
        (F.col('BLOOD PRODUCT COVID19 PROCESSING & ADMINISTRATION') == 1) |
        (F.col('BLOOD PRODUCTS COVID19') == 1),
        1
    ).otherwise(0).alias('BLOOD PRODUCTS'),
    
   F.when(
        #(F.col('EMERGENCY SERVICES COVID19') == 1) |
        (F.col('EMERGENCY SERVICES') == 1),
        1
    ).otherwise(0).alias('EMERGENCY SERVICES'))

chg_comb.show()

In [0]:
chg_fin = chg_comb.join(trans_chg_cov, on='PAT_KEY', how='left')\
    .drop('SMOKING DETERRENTS TOPICAL',
     'SMOKING DETERRENTS ORAL',
     'SMOKING DETERRENTS OTHER',
     'ANTIVIRALS, OTHER ORAL',
     'ANTIVIRALS, OTHER OTHER',
     'ANTIVIRALS, OTHER PARENTERAL',
     'IMMUNOSUP/TRANSPLANT AGENTS ORAL',
     'IMMUNOMODULATORS PARENTERAL',
     'IMMUNOSUP/TRANSPLANT AGENTS PARENTERAL',
     'IMMUNOSUP/TRANSPLANT AGENTS OTHER',
     'IMMUNOMODULATORS ORAL',
     'IMMUNOLOGIC AGENTS, MISC PARENTERAL',
     'IMMUNOLOGIC AGENTS, MISC OTHER',
     'VACCINES, PARENTERAL',
     'VACCINES, OTHER',
     'BLOOD PRODUCT PROCESSING & ADMINISTRATION',
     'BLOOD RELATED PRODUCTS PARENTERAL',
     'BLOOD PRODUCT COVID19 PROCESSING & ADMINISTRATION',
     'BLOOD PRODUCTS COVID19',
     'EMERGENCY SERVICES',
     'EMERGENCY SERVICES COVID19')


In [0]:
chg_fin.display()

In [0]:
chg_fin_2=prior_fo.join(chg_fin, on='PAT_KEY', how='left')


In [0]:

chg_fin_2.display()

CPT procedures

CCSR: 

Cardiovascular/ Renal: Circulatory system disorders (cardiomyopathies, cardiac arrhythmia, stroke – ischemis or unspecified, stroke – hemorrhagic), blood and hematologic disorders (pulmonary embolism, deep vein thrombosis, hematologic disorders), Renal disorders (chronic kidney disease) 

Neurologic: Mental disorders (Phychotic disorder, anxiety, depression or mood disorder), Sleep disorders, Neurological or nervous system disorders (ataxia/movement disorders, myoneural disorders, encephalitis, parkinsons or extrapyramidal syndromes, peripheral neuropathy, encephalopathy, autonomic dysfunction, amnesia/memory difficulty, dementia, seizures, delirium),  

Metabolic: Endocrine and metabolic disorders (Thyroid disese, diabetes type I or II),

Respiratory: Respiratory Disorders (hypoxemia, interstitial lung disease, pulmonary edema, abnormal breathing/dyspnea, respiratory symptoms/cough, chest/throat pain, bronchitis)

General: Symptoms (fever/malaise/fatigue, lymphadenopathy, weight loss, body ache/myalgia, headache, vertigo), 

Digestive: Gastrointestinal disorders (nausea/vomiting, changes in bowel habits, abdominal or pelvic pain)


PCC Conditions ICD 10

In [0]:
#post covid codes of interest for study. limiting paticd table to just these icd codes, fuzzy matching.

Pcc_codes=['E10', 'E11', 'E03', 'E06', 'I63', 'I64', 'I69', 'G45', 'G46', 'I60', 'I61', 'I62', 'R00', 'I47', 'I48', 'I49', 'I30', 'I40', 'I51.4', 'B33', 'I51.81', 'D72.81', 'D69', 'I82', 'I26', 'J84', 'J20', 'J40', 'J41', 'J42', 'J81', 'R06', 'R07', 'R05', 'R09.02', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F48', 'R45', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F38', 'F39', 'G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G61', 'G62', 'G64', 'G65', 'R40.0', 'R44', 'G40', 'G41', 'F01', 'F02', 'F03', 'G31', 'F05', 'G72', 'M60', 'G26', 'R26', 'R27', 'I95.1', 'G90', 'R55', 'G21', 'G24', 'G25', 'A85', 'A86', 'G04', 'G05', 'R29', 'R41', 'N18', 'N19', 'R10', 'K58', 'K59', 'A08', 'A09', 'R19.4', 'R19.7', 'R11', 'G43', 'G44', 'R51', 'M02', 'M25', 'M79', 'R50', 'R61', 'R53', 'G93.3', 'R59', 'R63', 'R64', 'A88', 'H81', 'R42', 'R20', 'B09', 'R21', 'R23', 'L63', 'L65', 'H90', 'H91', 'H92', 'H93', 'J31', 'R43', 'R13', 'H53', 'H54', 'G47', 'F51', 'U07.1', 'J12.82', 'B97.29', 'B34.2', 'M35.81', 'B94', 'U09.9']


pat_pcc_code = paticd.select(
    paticd.PAT_KEY.alias('PAT_KEY_COND'), 
    'ICD_CODE', 
    F.when(F.col("ICD_CODE").rlike("|".join(Pcc_codes)), 1).otherwise(0).alias('PCC_CODE'))\
.filter(F.col('PCC_CODE')==1)

#display(pat_pcc_code)

In [0]:


pat_cond1=prior_fo.join(pat_pcc_code, prior_fo['PAT_KEY'] == pat_pcc_code['PAT_KEY_COND'], how='left')\
    .select('MEDREC_KEY', 'PAT_KEY', 'ADMIT_DATE', 'PCC_CODE', 'ICD_CODE')\
        .distinct()

pat_cond1_1=pat_cond1.groupBy("MEDREC_KEY").agg(F.collect_list("ICD_CODE").alias("ICD_COV")).orderBy('MEDREC_KEY')


pat_cond2=prior_fo.join(pat_pcc_code, prior_fo['PAT_KEY_PR'] == pat_pcc_code['PAT_KEY_COND'], how='left')\
    .select('MEDREC_KEY', 'PAT_KEY_PR', 'ADMIT_DATE_PR',  'PCC_CODE', 'ICD_CODE')\
        .distinct()

pat_cond2_1=pat_cond2.groupBy("MEDREC_KEY").agg(F.collect_list("ICD_CODE").alias("ICD_PR")).orderBy('MEDREC_KEY')



pat_cond3=prior_fo.join(pat_pcc_code, prior_fo['PAT_KEY_FO'] == pat_pcc_code['PAT_KEY_COND'], how='left')\
    .select('MEDREC_KEY', 'PAT_KEY_FO', 'ADMIT_DATE_FO', 'PCC_CODE', 'ICD_CODE', 'FO_TIME')\
        .distinct()

pat_cond3_1=pat_cond3.groupBy("MEDREC_KEY", "FO_TIME").agg(F.collect_list("ICD_CODE").alias("ICD_FO")).orderBy('MEDREC_KEY')


pat_cond_fin=pat_cond1_1.join(pat_cond2_1, on='MEDREC_KEY', how='left')\
    .join(pat_cond3_1, on='MEDREC_KEY', how='left')\
    .select('MEDREC_KEY', 'ICD_COV', 'ICD_PR', 'ICD_FO', 'FO_TIME')\
    .orderBy('MEDREC_KEY')\
    .distinct()

pat_cond_fin.display()


In [0]:
con_count = pat_cond_fin.select('MEDREC_KEY').distinct().count()

print(con_count)

In [0]:
#Cardiovscular / Renal Disorder
card_ren = ['I63', 'I64', 'I69', 'G45', 'G46', 'I60', 'I61', 'I62', 'R00', 'I47', 'I48', 'I49', 'I30', 'I40', 'I51.4', 'B33', 'I51.81', 'D72.81', 'D69', 'I82', 'I26', 'N18', 'N19']

#Neurologic
neuro = ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G61', 'G62', 'G64', 'G65', 'R40.0', 'R44', 'G40', 'G41', 'F01', 'F02', 'F03', 'G31', 'F05', 'G72', 'M60', 'G26', 'R26', 'R27', 'I95.1', 'G90', 'R55', 'G21', 'G24', 'G25','A85', 'A86', 'G04', 'G05', 'R29', 'R41', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F48', 'R45', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F38', 'F39', 'G47', 'F51']

#Metabolic
metab = ['E10', 'E11', 'E03', 'E06']

#Respiratory
resp = ['J84', 'J20', 'J40', 'J41', 'J42', 'J81', 'R06', 'R07', 'R05', 'R09.02']

#General
gen = ['G43', 'G44', 'R51', 'M02', 'M25', 'M79', 'R50', 'R61', 'R53', 'G93.3', 'R59', 'R63', 'R64', 'A88', 'H81', 'R42']

#Digestive
dige = ['R10', 'K58', 'K59', 'A08', 'A09', 'R19.4', 'R19.7', 'R11']

#COVID
covid=['U07.1', 'J12.82', 'B97.29', 'B34.2']

In [0]:
icd_pr = pat_cond_fin.withColumn("ICD_COV_EXPLODED", F.explode("ICD_COV")) \
    .withColumn("ICD_PR_EXPLODED", F.explode("ICD_PR")) \
    .select('MEDREC_KEY', 'ICD_COV', 'ICD_PR',
    F.when((F.col("ICD_COV_EXPLODED").rlike("|".join(neuro))) | (F.col("ICD_PR_EXPLODED").rlike("|".join(neuro))), 1).otherwise(0).alias('NEURO_PR'),
    F.when((F.col("ICD_COV_EXPLODED").rlike("|".join(metab))) | (F.col("ICD_PR_EXPLODED").rlike("|".join(metab))), 1).otherwise(0).alias('METAB_PR'),
    F.when((F.col("ICD_COV_EXPLODED").rlike("|".join(resp))) | (F.col("ICD_PR_EXPLODED").rlike("|".join(resp))), 1).otherwise(0).alias('RESP_PR'),
    F.when((F.col("ICD_COV_EXPLODED").rlike("|".join(card_ren))) | (F.col("ICD_PR_EXPLODED").rlike("|".join(card_ren))), 1).otherwise(0).alias('CARD_REN_PR'),
    F.when((F.col("ICD_COV_EXPLODED").rlike("|".join(gen))) | (F.col("ICD_PR_EXPLODED").rlike("|".join(gen))), 1).otherwise(0).alias('GEN_PR'),
    F.when((F.col("ICD_COV_EXPLODED").rlike("|".join(dige))) | (F.col("ICD_PR_EXPLODED").rlike("|".join(dige))), 1).otherwise(0).alias('DIGE_PR'),
    F.when((F.col("ICD_COV_EXPLODED").rlike("|".join(covid))) | (F.col("ICD_PR_EXPLODED").rlike("|".join(covid))), 1).otherwise(0).alias('COVID_PR')).distinct()

In [0]:
icd_fo = pat_cond_fin.withColumn("ICD_FO_EXPLODED", F.explode("ICD_FO")) \
    .select('MEDREC_KEY', 'ICD_FO', 'FO_TIME',
    F.when((F.col("ICD_FO_EXPLODED").rlike("|".join(neuro))), 1).otherwise(0).alias('NEURO_FO'),
    F.when((F.col("ICD_FO_EXPLODED").rlike("|".join(metab))), 1).otherwise(0).alias('METAB_FO'),
    F.when((F.col("ICD_FO_EXPLODED").rlike("|".join(resp))), 1).otherwise(0).alias('RESP_FO'),
    F.when((F.col("ICD_FO_EXPLODED").rlike("|".join(card_ren))), 1).otherwise(0).alias('CARD_REN_FO'),
    F.when((F.col("ICD_FO_EXPLODED").rlike("|".join(gen))), 1).otherwise(0).alias('GEN_FO'),
    F.when((F.col("ICD_FO_EXPLODED").rlike("|".join(dige))), 1).otherwise(0).alias('DIGE_FO'),
    F.when((F.col("ICD_FO_EXPLODED").rlike("|".join(covid))), 1).otherwise(0).alias('COVID_FO')).distinct()

In [0]:
icd_fo.display()

In [0]:
icd_pr1=icd_pr.groupBy('MEDREC_KEY').agg(
F.max('NEURO_PR').alias('NEURO_PR'),
F.max('METAB_PR').alias('METAB_PR'),
F.max('RESP_PR').alias('RESP_PR'),
F.max('CARD_REN_PR').alias('CARD_REN_PR'),
F.max('GEN_PR').alias('GEN_PR'),
F.max('DIGE_PR').alias('DIGE_PR'),
F.max('COVID_PR').alias('COVID_PR'))

In [0]:
icd_fo1=icd_fo.groupBy('MEDREC_KEY', 'FO_TIME').agg(
F.max('NEURO_FO').alias('NEURO_FO'),
F.max('METAB_FO').alias('METAB_FO'),
F.max('RESP_FO').alias('RESP_FO'),
F.max('CARD_REN_FO').alias('CARD_REN_FO'),
F.max('GEN_FO').alias('GEN_FO'),
F.max('DIGE_FO').alias('DIGE_FO'),
F.max('COVID_FO').alias('COVID_FO'))

In [0]:
icd_fo1.display()

In [0]:
icd_fin=icd_pr1.join(icd_fo1, on='MEDREC_KEY', how='LEFT').orderBy('MEDREC_KEY')

#icd_fin.display()

In [0]:
prior_fo_fin=chg_fin_2.join(icd_fin, on='MEDREC_KEY', how='LEFT')

In [0]:
prior_fo_fin=chg_fin_2.join(icd_fin, on='MEDREC_KEY', how='LEFT')\
.drop('PAT_KEY', 'ADMIT_DATE', 'DISCHARGE_DATE', 'PROV_ID', 'I_O_IND', 'PAT_TYPE', 'INP', 'COV_COUNT', 'PAT_KEY_PR', 'ADMIT_DATE_PR', 'YEAR_PRIOR',
'PRIOR_YR',
'INPATIENT',
'PAT_KEY_FO',
'ADMIT_DATE_FO',
'YEAR_FO',
'FO_YR',
'PRO FEE', chg_fin_2.FO_TIME)\
.orderBy('MEDREC_KEY')\
.distinct()


prior_fo_fin.display()

In [0]:
fin_count = prior_fo_fin.select('MEDREC_KEY').distinct().count()

print(fin_count)

In [0]:
def clean_column_name(name):
    # Keep only alphanumeric characters and underscores
    return re.sub(r'[^a-zA-Z0-9_]', '_', name)

cleaned_columns = [clean_column_name(col) for col in prior_fo_fin.columns]
prior_fo_fin_cleaned = prior_fo_fin.toDF(*cleaned_columns)

prior_fo_fin_cleaned=prior_fo_fin_cleaned.fillna(0)

In [0]:
prior_fo_fin_cleaned.orderBy('MEDREC_KEY').display()

In [0]:
(prior_fo_fin_cleaned
 .coalesce(1)
 .write
 .format('delta')
 .mode('overwrite')
 .saveAsTable('cdh_premier_ml.Bart_Pcc_2')
)